### To run the cbgt network, see first the technical_testbench to install all the required packages and start the server 

In [1]:
#Auto reloading 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload 

In [ ]:
#Removing indentation issues:
!autopep8 --in-place --aggressive --aggressive init_params.py
!autopep8 --in-place --aggressive --aggressive popconstruct.py
!autopep8 --in-place --aggressive --aggressive generateepochs.py
!autopep8 --in-place --aggressive --aggressive qvalues.py

In [2]:
!python setup.py build_ext --inplace

Compiling agent_timestep.pyx because it changed.
[1/1] Cythonizing agent_timestep.pyx
running build_ext
building 'agent_timestep' extension
C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.29.30133\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\Users\mclapp\anaconda3\envs\cbgt\include -IC:\Users\mclapp\anaconda3\envs\cbgt\include "-IC:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.29.30133\ATLMFC\include" "-IC:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.29.30133\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\shared" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\winrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\cppwinrt" /Tcagent_timestep.c /Fobuild\temp.win-amd64-3.8\Release\agent_ti

C:\Users\mclapp\anaconda3\envs\cbgt\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\mclapp\Downloads\cbgt-final\agent_timestep.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [2]:
#Importing scripts:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct as popconstruct
import qvalues as qval
import generateepochs as gen
import pdb

2021-08-25 09:18:28,706	INFO worker.py:800 -- Connecting to existing Ray cluster at address: 192.168.1.201:6379


In [3]:
from agentmatrixinit import *

In [4]:
from general_init_codeblocks import *

In [5]:
from agent_timestep import timestep_mutator, multitimestep_mutator

# Create pipelines

### Reward pipeline

In [6]:
rsg = cbgt.Pipeline() #rsg is short for 'reward schedule generator'


(rsg.volatile_pattern,
 rsg.cp_idx,
 rsg.cp_indicator,
 rsg.noisy_pattern,
 rsg.t1_epochs,
 rsg.t2_epochs,
) = rsg[gen.GenRewardSchedule](
        rsg.n_trials,
        rsg.volatility,
        rsg.conflict,
        rsg.reward_mu,
        rsg.reward_std,
    ).shape(6)

### Network pipeline

In [7]:
#Defining necessary codeblocks:

#MODIFIERS:

#init_params.py: to modify the neuronal default values

def codeblock_modifycelldefaults(self):
    self.celldefaults = par.helper_cellparams(self.params)

def codeblock_modifypopspecific(self):
    self.popspecific = par.helper_popspecific(self.pops)

def codeblock_modifyreceptordefaults(self):
    self.receptordefaults = par.helper_receptor(self.receps)

def codeblock_modifybasestim(self):
    self.basestim = par.helper_basestim(self.base)

def codeblock_modifydpmndefaults(self):
    self.dpmndefaults = par.helper_dpmn(self.dpmns)

def codeblock_modifyd1defaults(self):
    self.d1defaults = par.helper_d1(self.d1)

def codeblock_modifyd2defaults(self):
    self.d2defaults = par.helper_d2(self.d2)

def codeblock_modifyactionchannels(self):
    self.actionchannels = par.helper_actionchannels(self.channels)

#popconstruct.py: to modify population parameters

def codeblock_popconstruct(self):
    self.popdata = popconstruct.helper_popconstruct(self.actionchannels, self.popspecific, self.celldefaults, self.receptordefaults, self.basestim, self.dpmndefaults, self.d1defaults, self.d2defaults)

def codeblock_poppathways(self):
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways)

#init_params.py: Q-values initialization and update

def codeblock_init_Q_support_params(self):
    self.Q_support_params = qval.helper_init_Q_support_params()

def codeblock_update_Q_support_params(self,reward_val, chosen_action):
    self.Q_support_params = qval.helper_update_Q_support_params(self.Q_support_params,self.reward_val,pl.chosen_action)

def codeblock_init_Q_df(self):
    self.Q_df = qval.helper_init_Q_df(self.channels)

def codeblock_update_Q_df(self):
    self.Q_df, self.Q_support_params, self.dpmndefaults = qval.helper_update_Q_df(self.Q_df, self.Q_support_params, self.dpmndefaults,pl.trial_num)


In [8]:
pl = cbgt.Pipeline()

In [9]:
#Adding rsg pipeline to the network pipeline: 
pl.add(rsg) 

In [10]:
#to update the Q-values 
pl.trial_num = 0 #first row of Q-values df - initialization data 
pl.chosen_action = None # 2 #chosen action for the current trial

In [11]:
#Defining necessary function modules: 

#init_params.py: default neuronal values 
pl.celldefaults = par.helper_cellparams()
pl.popspecific = par.helper_popspecific()
pl.receptordefaults = par.helper_receptor()
pl.basestim = par.helper_basestim()
pl.dpmndefaults = par.helper_dpmn()
pl.d1defaults = par.helper_d1()
pl.d2defaults = par.helper_d2()
#pl.actionchannels = pl[par.helper_actionchannels]()

#popconstruct.py: default population parameters 
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)

#popconstruct.py: to create connectivity grids
pl.connectivity_AMPA, pl.meaneff_AMPA, pl.plastic_AMPA = pl[popconstruct.helper_connectivity]('AMPA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_GABA, pl.meaneff_GABA, pl.plastic_GABA = pl[popconstruct.helper_connectivity]('GABA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_NMDA, pl.meaneff_NMDA, pl.plastic_NMDA = pl[popconstruct.helper_connectivity]('NMDA', pl.popdata, pl.pathways).shape(3)

In [12]:
#Adding codeblocks to the newtork pipeline: 
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct)
pl.add(codeblock_poppathways)

### Q-values pipeline

In [13]:
#Creating a new pipeline for Q-values: 
q_val_pipe = cbgt.Pipeline()

In [14]:
#Defining necessary function modules: 

#qvalues.py
q_val_pipe.Q_support_params = q_val_pipe[qval.helper_init_Q_support_params]()
q_val_pipe.Q_df = q_val_pipe[qval.helper_init_Q_df](pl.actionchannels)

#rsg.reward_val = q_val_pipe[qval.get_reward_value](rsg.t1_epochs,rsg.t2_epochs,pl.chosen_action,pl.trial_num) 
#q_val_pipe.Q_support_params = q_val_pipe[qval.helper_update_Q_support_params](q_val_pipe.Q_support_params,rsg.reward_val,pl.chosen_action)
#(q_val_pipe.Q_df, q_val_pipe.Q_support_params, pl.dpmndefaults) = q_val_pipe[qval.helper_update_Q_df](q_val_pipe.Q_df,q_val_pipe.Q_support_params,pl.dpmndefaults,pl.trial_num).shape(3)

In [15]:
#Adding the q_val_pipe to the main network pipeline pl:
pl.add(q_val_pipe)

# Timestep loop

In [16]:
def mega_loop(slf):
    slf.AMPA_con,slf.AMPA_eff = CreateSynapses(slf.popdata,slf.connectivity_AMPA,slf.meaneff_AMPA,slf.plastic_AMPA)
    slf.GABA_con,slf.GABA_eff = CreateSynapses(slf.popdata,slf.connectivity_GABA,slf.meaneff_GABA,slf.plastic_GABA)
    slf.NMDA_con,slf.NMDA_eff = CreateSynapses(slf.popdata,slf.connectivity_NMDA,slf.meaneff_NMDA,slf.plastic_NMDA)

    popdata = slf.popdata
    actionchannels = slf.actionchannels
    agent = initializeAgent(popdata)
    slf.agent = agent

    agent.AMPA_con,agent.AMPA_eff = slf.AMPA_con,slf.AMPA_eff
    agent.GABA_con,agent.GABA_eff = slf.GABA_con,slf.GABA_eff
    agent.NMDA_con,agent.NMDA_eff = slf.NMDA_con,slf.NMDA_eff
    agent.LastConductanceNMDA = CreateAuxiliarySynapseData(popdata,slf.connectivity_NMDA)

    agent.FRs = agent.rollingbuffer.mean(1) / untrace(list(popdata['N'])) / agent.dt * 1000

    multitimestep_mutator(agent,popdata,1000)

    agent.phase = 0
    agent.phasetimer = 0
    agent.motor_queued = None
    agent.dpmn_queued = None
    agent.gain = np.ones(len(actionchannels))
    agent.extstim = np.zeros(len(actionchannels))
    agent.ramping_extstim = np.zeros(len(actionchannels))
    agent.in_popids = np.where(popdata['name'] == 'LIP')[0]
    agent.out_popids = np.where(popdata['name'] == 'Th')[0]

    presented_stimulus = 1
    slf.chosen_action = None

    while slf.trial_num < slf.n_trials:
        agent.extstim = agent.gain * presented_stimulus * 4.0  # TODO: make 3.0 a param
        agent.ramping_extstim = agent.ramping_extstim * 0.9 + agent.extstim * 0.1
        for action_idx in range(len(actionchannels)):
            popid = agent.in_popids[action_idx]
            agent.FreqExt_AMPA[popid] = np.ones(len(agent.FreqExt_AMPA[popid])) * agent.ramping_extstim[action_idx]
        multitimestep_mutator(agent,popdata,5)
        agent.phasetimer += 1 # 1 ms = 5 * dt
        #agent.FRs = np.stack((agent.FRs,agent.rollingbuffer.mean(1) / untrace(list(popdata['N'])) / agent.dt * 1000))

        if agent.phase == 0:
            gateFRs = agent.rollingbuffer[agent.out_popids].mean(1) / untrace(list(popdata['N'][agent.out_popids])) / agent.dt * 1000
            thresholds_crossed = np.where(gateFRs > 30)[0]
            if len(thresholds_crossed) > 0 or agent.phasetimer > 1000:
                agent.phase = 1
                agent.phasetimer = 0
                agent.gain = np.zeros(len(actionchannels))
                if len(thresholds_crossed) > 0:
                    agent.motor_queued = thresholds_crossed[0]
                    agent.gain[agent.motor_queued] = 0.5
                else:
                    agent.motor_queued = -1

        if agent.phase == 1:
            if agent.phasetimer > 100:
                agent.phase = 2
                agent.phasetimer = 0
                agent.gain = np.zeros(len(actionchannels))
                slf.chosen_action = untrace(actionchannels.iloc[agent.motor_queued,0])
                agent.motor_queued = None

        if agent.phase == 2:
            if agent.phasetimer > 100:
                slf.dpmndefaults['dpmn_DAp'] = 0
                slf.trial_num += 1
                agent.phase = 0
                agent.phasetimer = 0


        # environment

        if slf.chosen_action is not None:
            slf.reward_val = qval.get_reward_value(slf.t1_epochs,slf.t2_epochs,slf.chosen_action,slf.trial_num)
            slf.Q_support_params = qval.helper_update_Q_support_params(slf.Q_support_params,slf.reward_val,slf.chosen_action)
            (slf.Q_df, slf.Q_support_params, slf.dpmndefaults) = qval.helper_update_Q_df(slf.Q_df,slf.Q_support_params,slf.dpmndefaults,slf.trial_num)
            slf.chosen_action = None

In [17]:
pl.add(mega_loop)

# Running the pipeline

### 1. Default values

In [18]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
environment = {
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : None, #action channels related parameters (init_params.py)
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df':None, #initialized Q-values df (qvalues.py)  
    'n_trials': 5, #number of trials (generateepochs.py)
    'volatility': 2, #frequency of changepoints (generateepochs.py)
    'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment)

(pid=23824) 2 4 0.161668`
(pid=23824) 3 5 0.161668`
(pid=23824) 10 6 1.0`
(pid=23824) 10 8 1.0`
(pid=23824) 10 10 0.4335`
(pid=23824) 10 11 0.4335`
(pid=23824) 10 12 1.0`
(pid=23824) 10 13 1.0`
(pid=23824) 10 14 1.0`
(pid=23824) 10 15 0.241667`
(pid=23824) 11 7 1.0`
(pid=23824) 11 9 1.0`
(pid=23824) 11 10 0.4335`
(pid=23824) 11 11 0.4335`
(pid=23824) 11 12 1.0`
(pid=23824) 11 13 1.0`
(pid=23824) 11 14 1.0`
(pid=23824) 11 15 0.241667`
(pid=23824) 12 6 1.0`
(pid=23824) 12 8 1.0`
(pid=23824) 12 14 0.8334`
(pid=23824) 13 7 1.0`
(pid=23824) 13 9 1.0`
(pid=23824) 13 14 0.8334`
(pid=23824) 0 12 1.0`
(pid=23824) 1 13 1.0`
(pid=23824) 4 0 1.0`
(pid=23824) 4 2 0.0667`
(pid=23824) 4 4 0.0667`
(pid=23824) 4 5 0.0667`
(pid=23824) 5 1 1.0`
(pid=23824) 5 3 0.0667`
(pid=23824) 5 4 0.0667`
(pid=23824) 5 5 0.0667`
(pid=23824) 6 0 1.0`
(pid=23824) 6 6 0.45`
(pid=23824) 6 8 0.45`
(pid=23824) 7 1 1.0`
(pid=23824) 7 7 0.45`
(pid=23824) 7 9 0.45`
(pid=23824) 8 4 1.0`
(pid=23824) 8 6 0.5`
(pid=23824) 8 8 0.45

In [19]:
results['Q_df']

,1,2
0,0.500000,0.500000
0,0.644218,0.500000
0,0.644218,0.275000
0,0.644218,0.151250
0,0.644218,0.579087
0,0.644218,0.318498


2021-08-26 01:52:51,220	ERROR import_thread.py:88 -- ImportThread: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
2021-08-26 01:52:51,235	ERROR worker.py:468 -- print_logs: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
2021-08-26 01:52:51,236	ERROR worker.py:1191 -- listen_error_messages_raylet: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)


In [ ]:
dev = cbgt.Pipeline()

In [ ]:
dev.add(codeblock_agent_phase0)

In [ ]:
output = cbgt.ExecutionManager(cores=7).run(dev,results) 